<a href="https://colab.research.google.com/github/PeterZuckerwall/Data-Science/blob/main/fcc_book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [1]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-02 07:28:20--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   105MB/s    in 0.2s    

2025-03-02 07:28:21 (105 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [5]:



df_books = pd.read_csv(
    'BX-Books.csv',
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'}
)

df_ratings = pd.read_csv(
    'BX-Book-Ratings.csv',
    encoding="ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'}
)



df = pd.merge(df_ratings, df_books, on='isbn')


rating_counts = df.groupby('title').size()
popular_books = rating_counts[rating_counts > 50].index
df = df[df['title'].isin(popular_books)]
book_user_matrix = df.pivot_table(index='title', columns='user', values='rating', fill_value=0)
sparse_matrix = csr_matrix(book_user_matrix.values)

model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
model_knn.fit(sparse_matrix)

def get_recommends(book_title):

    if book_title not in book_user_matrix.index:
        raise ValueError(f"Book '{book_title}' not found in the dataset.")


    book_index = book_user_matrix.index.get_loc(book_title)

    distances, indices = model_knn.kneighbors(sparse_matrix[book_index], n_neighbors=6)

    recommendations = []
    for i in range(1, len(distances[0])):
        neighbor_index = indices[0][i]
        neighbor_title = book_user_matrix.index[neighbor_index]
        distance = distances[0][i]
        recommendations.append([neighbor_title, distance])


    return [book_title, recommendations]


In [6]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Lovely Bones: A Novel', 0.90959823], ['Blue Diary', 0.9101138], ['I Know This Much Is True', 0.9102433], ["The Pilot's Wife : A Novel", 0.9172571], ['The Weight of Water', 0.9210188]]]
You haven't passed yet. Keep trying!
